https://www.tensorflow.org/tutorials/keras/save_and_load

### Save and load models

Model progress can be saved during and after training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

<ul>
    <li>code to create the model, and</li>
    <li>the trained weights, or parameters, for the model</li>
</ul>

Sharing this data helps others understand how the model works and try it themselves with new data.

### Options

There are different ways to save TensorFlow models depending on the API you're using. This guide uses <a href="https://www.tensorflow.org/guide/keras">tf.keras</a>—a high-level API to build and train models in TensorFlow. The new, high-level .keras format used in this tutorial is recommended for saving Keras objects, as it provides robust, efficient name-based saving that is often easier to debug than low-level or legacy formats. For more advanced saving or serialization workflows, especially those involving custom objects, please refer to the <a href="https://www.tensorflow.org/guide/keras/save_and_serialize">Save and load Keras models guide</a>. For other approaches, refer to the <a href="https://www.tensorflow.org/guide/saved_model">Using the SavedModel format guide</a>.

### Setup

#### Installs and imports

Install and import TensorFlow and dependencies:

In [2]:
!pip install pyyaml h5py  # Required to save models in HDF5 format


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2024-05-31 03:12:06.822049: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.15.0


### Get an example dataset

To demonstrate how to save and load weights, you'll use the <a href="http://yann.lecun.com/exdb/mnist/">MNIST dataset</a>. To speed up these runs, use the first 1000 examples:

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 1s 0us/step


### Define a model
Start by building a simple sequential model:

In [5]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Save checkpoints during training

You can use a trained model without having to retrain it, or pick-up training where you left off in case the training process was interrupted. The <a href="https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint">tf.keras.callbacks.ModelCheckpoint</a> callback allows you to continually save the model both during and at the end of training.

### Checkpoint callback usage

Create a <a href="https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint">tf.keras.callbacks.ModelCheckpoint</a> callback that saves weights only during training:

In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
25/32 [======================>.......] - ETA: 0s - loss: 1.2749 - sparse_categorical_accuracy: 0.6463
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 12ms/step - loss: 1.1412 - sparse_categorical_accuracy: 0.6870 - val_loss: 0.7272 - val_sparse_categorical_accuracy: 0.7770
Epoch 2/10
29/32 [==========================>...] - ETA: 0s - loss: 0.4331 - sparse_categorical_accuracy: 0.8804
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.4216 - sparse_categorical_accuracy: 0.8820 - val_loss: 0.5347 - val_sparse_categorical_accuracy: 0.8380
Epoch 3/10
28/32 [=========================>....] - ETA: 0s - loss: 0.2758 - sparse_categorical_accuracy: 0.9308
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.2808 - sparse_categorical_accuracy: 0.9250 - val_loss: 0.4581 - val_sparse_categorical_accuracy: 0.8500
Epoch 4/10
24/32 [=========

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [7]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.index', 'cp.ckpt.data-00000-of-00001']

As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights.

Now rebuild a fresh, untrained model and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [8]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3038 - sparse_categorical_accuracy: 0.1400 - 136ms/epoch - 4ms/step
Untrained model, accuracy: 14.00%


Then load the weights from the checkpoint and re-evaluate:

In [9]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4153 - sparse_categorical_accuracy: 0.8690 - 55ms/epoch - 2ms/step
Restored model, accuracy: 86.90%


Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [10]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*n_batches)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


Now, review the resulting checkpoints and choose the latest one:

In [11]:
os.listdir(checkpoint_dir)

['cp-0045.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0000.ckpt.index',
 'checkpoint',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001']

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

To test, reset the model, and load the latest checkpoint:

In [13]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.5008 - sparse_categorical_accuracy: 0.8700 - 123ms/epoch - 4ms/step
Restored model, accuracy: 87.00%


### What are these files?

The above code stores the weights to a collection of <a href="https://www.tensorflow.org/guide/checkpoint">checkpoint</a>-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

<ul>
    <li>One or more shards that contain your model's weights.</li>
    <li>An index file that indicates which weights are stored in which shard.</li>
</ul>

If you are training a model on a single machine, you'll have one shard with the suffix: .data-00000-of-00001

### Manually save weights

To save weights manually, use <a href="https://www.tensorflow.org/api_docs/python/tf/keras/Model#save_weights">tf.keras.Model.save_weights</a>. By default, <a href="https://www.tensorflow.org/api_docs/python/tf/keras">tf.keras</a>—and the <a href="https://www.tensorflow.org/api_docs/python/tf/keras/Model#save_weights">Model.save_weights</a> method in particular—uses the TensorFlow <a href="https://www.tensorflow.org/guide/checkpoint">Checkpoint</a> format with a .ckpt extension. To save in the HDF5 format with a .h5 extension, refer to the <a href="https://www.tensorflow.org/guide/keras/save_and_serialize">Save and load models</a> guide.

In [14]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.5008 - sparse_categorical_accuracy: 0.8700 - 116ms/epoch - 4ms/step
Restored model, accuracy: 87.00%


### Save the entire model

Call <a href="https://www.tensorflow.org/api_docs/python/tf/keras/Model#save">tf.keras.Model.save</a> to save a model's architecture, weights, and training configuration in a single model.keras zip archive.

An entire model can be saved in three different file formats (the new .keras format and two legacy formats: SavedModel, and HDF5). Saving a model as path/to/model.keras automatically saves in the latest format.

You can switch to the SavedModel format by:

<ul>
    <li>Passing save_format='tf' to save()</li>
    <li>Passing a filename without an extension</li>
</ul>

You can switch to the H5 format by:

<ul>
    <li>Passing save_format='h5' to save()</li>
    <li>Passing a filename that ends in .h5</li>
</ul>

Saving a fully-functional model is very useful—you can load them in TensorFlow.js (<a href="https://www.tensorflow.org/js/tutorials/conversion/import_saved_model">Saved Model</a>, <a href="https://www.tensorflow.org/js/tutorials/conversion/import_keras">HDF5</a>) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite (<a href="https://www.tensorflow.org/lite/models/convert/#convert_a_savedmodel_recommended_">Saved Model</a>, <a href="https://www.tensorflow.org/lite/models/convert/#convert_a_keras_model_">HDF5</a>)

*Custom objects (for example, subclassed models or layers) require special attention when saving and loading. Refer to the Saving custom objects section below.

### New high-level .keras format

The new Keras v3 saving format, marked by the .keras extension, is a more simple, efficient format that implements name-based saving, ensuring what you load is exactly what you saved, from Python's perspective. This makes debugging much easier, and it is the recommended format for Keras.

The section below illustrates how to save and restore the model in the .keras format.

In [15]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

Epoch 1/5
32/32 [==============================] - 1s 4ms/step - loss: 1.1601 - sparse_categorical_accuracy: 0.6570
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4195 - sparse_categorical_accuracy: 0.8870
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2768 - sparse_categorical_accuracy: 0.9300
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2079 - sparse_categorical_accuracy: 0.9530
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1555 - sparse_categorical_accuracy: 0.9650


Reload a fresh Keras model from the .keras zip archive:

In [16]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Try running evaluate and predict with the loaded model:

In [17]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4159 - sparse_categorical_accuracy: 0.8660 - 135ms/epoch - 4ms/step
Restored model, accuracy: 86.60%
32/32 [==============================] - 0s 1ms/step
(1000, 10)


### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using <a href="https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model">tf.keras.models.load_model</a> and are compatible with TensorFlow Serving. The <a href="https://www.tensorflow.org/guide/saved_model">SavedModel guide</a> goes into detail about how to serve/inspect the SavedModel. The section below illustrates the steps to save and restore the model.

In [18]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 4ms/step - loss: 1.1898 - sparse_categorical_accuracy: 0.6630
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4549 - sparse_categorical_accuracy: 0.8720
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2956 - sparse_categorical_accuracy: 0.9170
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2191 - sparse_categorical_accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1611 - sparse_categorical_accuracy: 0.9650
INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


The SavedModel format is a directory containing a protobuf binary and a TensorFlow checkpoint. Inspect the saved model directory:

In [19]:
# my_model directory
!ls saved_model

my_model


In [20]:
# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


Reload a fresh Keras model from the saved model:

In [21]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [22]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4249 - sparse_categorical_accuracy: 0.8640 - 135ms/epoch - 4ms/step
Restored model, accuracy: 86.40%
32/32 [==============================] - 0s 1ms/step
(1000, 10)


### HDF5 format

Keras provides a basic legacy high-level save format using the <a href="https://en.wikipedia.org/wiki/Hierarchical_Data_Format">HDF5</a> standard. 

In [23]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1639 - sparse_categorical_accuracy: 0.6690
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4150 - sparse_categorical_accuracy: 0.8930
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2933 - sparse_categorical_accuracy: 0.9210
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2066 - sparse_categorical_accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1450 - sparse_categorical_accuracy: 0.9740


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Now, recreate the model from that file:

In [24]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Check its accuracy:

In [25]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4258 - sparse_categorical_accuracy: 0.8650 - 134ms/epoch - 4ms/step
Restored model, accuracy: 86.50%


Keras saves models by inspecting their architectures. This technique saves everything:

<ul>
    <li>The weight values</li>
    <li>The model's architecture</li>
    <li>The model's training configuration (what you pass to the .compile() method)</li>
    <li>The optimizer and its state, if any (this enables you to restart training where you left off)</li>
</ul>

Keras is not able to save the v1.x optimizers (from <a href="https://www.tensorflow.org/api_docs/python/tf/compat/v1/train">tf.compat.v1.train</a>) since they aren't compatible with checkpoints. For v1.x optimizers, you need to re-compile the model after loading—losing the state of the optimizer.

### Saving custom objects

If you are using the SavedModel format, you can skip this section. The key difference between high-level .keras/HDF5 formats and the low-level SavedModel format is that the .keras/HDF5 formats uses object configs to save the model architecture, while SavedModel saves the execution graph. Thus, SavedModels are able to save custom objects like subclassed models and custom layers without requiring the original code. However, debugging low-level SavedModels can be more difficult as a result, and we recommend using the high-level .keras format instead due to its name-based, Keras-native nature.

To save custom objects to .keras and HDF5, you must do the following:

<ol>
    <li>
        Define a get_config method in your object, and optionally a from_config classmethod.
        <ul>
            <li>get_config(self) returns a JSON-serializable dictionary of parameters needed to recreate the object.</li>
            <li>from_config(cls, config) uses the returned config from get_config to create a new object. By default, this function will use the config as initialization kwargs (return cls(**config)).</li>
        </ul>
    </li>
    <li>
        Pass the custom objects to the model in one of three ways:
        <ul>
            <li>Register the custom object with the <a href="https://www.tensorflow.org/api_docs/python/tf/keras/utils/register_keras_serializable">@tf.keras.utils.register_keras_serializable</a> decorator. (recommended)</li>
            <li>Directly pass the object to the custom_objects argument when loading the model. The argument must be a dictionary mapping the string class name to the Python class. E.g., tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})</li>
            <li>Use a <a href="https://www.tensorflow.org/api_docs/python/tf/keras/utils/CustomObjectScope">tf.keras.utils.custom_object_scope</a> with the object included in the custom_objects dictionary argument, and place a <a href="https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model">tf.keras.models.load_model(path)</a> call within the scope.</li>
        </ul>
    </li>
</ol>

Refer to the <a href="https://www.tensorflow.org/guide/keras/custom_layers_and_models">Writing layers and models from scratch</a> tutorial for examples of custom objects and get_config.